In [ ]:
import sys, unittest, time, datetime
import urllib.request, urllib.error, urllib.parse
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import InvalidArgumentException
from tqdm.notebook import tqdm

import pymongo
from pymongo.errors import DuplicateKeyError
from pymongo import MongoClient
import traceback
import pickle

In [ ]:
client = MongoClient()
db = client.youtube
collection = db.popscience

In [ ]:
with open('youtube_links_list.txt') as f:
    youtube_links_list = f.read().splitlines()

# Скачивание информации не вошедшей в плайлисты без повторов

In [ ]:
%%time

sleep = 3
URLError_counter = 0

for url in tqdm(youtube_links_list):
    try:
        find_result = collection.find_one({"origin_url": url})
        if find_result is not None:
            continue

        info = {}
        try:
            yt = YouTube(url)
        except VideoPrivate:
            print(f"Private video {url}")
            continue
        except VideoUnavailable:
            print(f"Unavailable video {url}")
            continue
        info["title"] = yt.title
        info["playlist_title"] = ''
        info["playlist_description"] = ''
        info["playlist_views"] = ''
        info["date"] = yt.publish_date
        info["origin_url"] = url
        info["views"] = yt.views
        info["duation"] = yt.length
        info["author"] = yt.author
        info["tags"] = yt.keywords
        info["about"] = yt.description
        info["rating"] = yt.rating
        info["platform"] = "YouTube"
        info["theme"] = yt.thumbnail_url
        if yt.captions.get("ru") is not None:
            caption = yt.captions.get('ru')
            info["language"] = "ru"
        elif yt.captions.get("a.ru") is not None:
            caption = yt.captions.get('a.ru')
            info["language"] = "a.ru"
        if caption:
            info["captions_xml"] = caption.xml_captions
            info["captions_str"] = caption.generate_srt_captions()
        result = collection.insert_one(info)
    except DuplicateKeyError:
        print("============ DuplicateKeyError ============")
        continue
    except URLError:
        print("============ URLError ============")
        traceback.print_exc()
#         URLError_counter += 1
#         if URLError_counter == 20:
#             URLError_counter = 0
#             break
        time.sleep(sleep)
        continue
    except KeyError:
        print("============ KeyError ============")
        continue
    except KeyboardInterrupt:
        print("============ KeyboardInterrupt ============")
        break
    except:
        print("============ UnnounError ============")
        traceback.print_exc()
        continue
#     else:
#         print("============ ELSE ============")
#         break